In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
html_data=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup=BeautifulSoup(html_data, 'lxml')
print(soup.title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# Dataframe containing 3 columns of Toronto City Data

In [5]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [6]:
# Group the df by postal code and borough for each neighborhood

df_postcode = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_postcode.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
# Joint the df_postcode with CSV lon&Lat columns to create a new data frame call toronto_data

locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
toronto_data = df_postcode.join(locgeo_df, on='PostalCode')

# Dataframe containing Longitude and Latitude of Toronto neighborhood

In [8]:
toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# Explore and Cluster the neighborhoods in Toronto

## A. Create a cluster map of Toronto based on boroughs and bring in venue data

In [128]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [133]:
toronto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PostalCode    103 non-null    object 
 1   Borough       103 non-null    object 
 2   Neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.1+ KB


### A1. Find latitude and longtitude and draw the Toronto Map

In [98]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent='canada_explorer')
location = geolocator.geocode(address)
latitude = location.latitude #43.6532
longitude = location.longitude # -79.3832
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


The geographical coordinate of Toronto are 43.6534817, -79.3839347.


In [134]:
Ncount = toronto_data['PostalCode'].count()
print('The toronto_data has {} postal code and {} borough'.format(
          len(toronto_data['PostalCode'].unique()),
          len(toronto_data['Borough'].unique()),
     )
     )

The toronto_data has 103 postal code and 15 borough


### A2. Visualize Toronto neighborhood represented by postal code

In [234]:
for lat, lng, postalcode, in zip(toronto_data['Latitude'], toronto_data['Longitude'],
                                          toronto_data['PostalCode']):
    label = '{}'.format(postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=1,
    popup=label,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto                                                  

### A2a Visualize Scarborough in Toronto

In [250]:
scarborough_data= toronto_data[toronto_data['Borough']=='Scarborough'].reset_index(drop=True)
scarborough_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### A2b Find longitudue and latitude of Scarborough

In [251]:
address = 'Scarborough, Canada'

geolocator = Nominatim(user_agent='scarborough_explorer')
location = geolocator.geocode(address)
latitude = location.latitude 
longitude = location.longitude 
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.7729744, -79.2576479.


### A2c Visualize Scarborough neighborhood

In [252]:
# create map of Manhattan using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], 
                           scarborough_data['PostalCode']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### A3. Establish Foursquare Credentials and version for API

In [253]:
CLIENT_ID = 'IPFKGSTEYOILEGG2MJRFNKDX2YV5WEH44MR2KZDKZOWFBXWM' # your Foursquare ID
CLIENT_SECRET = '21MX2NLSSB4PWVFR3Y04NYJJOQCNDEWTM0TMVVLVKXK2B3CD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IPFKGSTEYOILEGG2MJRFNKDX2YV5WEH44MR2KZDKZOWFBXWM
CLIENT_SECRET:21MX2NLSSB4PWVFR3Y04NYJJOQCNDEWTM0TMVVLVKXK2B3CD


### A4. Get the name of the first neighborhood and it's latitude and longitude

In [254]:
scarborough_data_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
scarborough_data_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

scarborough_data_name = scarborough_data.loc[0, 'Borough'] # neighborhood postal code

print('Latitude and longitude values of {} are {}, {}.'.format(scarborough_data_name, 
                                                               scarborough_data_latitude, 
                                                               scarborough_data_longitude))

Latitude and longitude values of Scarborough are 43.806686299999996, -79.19435340000001.


### A5. Get the top 100 venues in Scarborough within radius of 700 meters

In [255]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 700 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    scarborough_data_latitude, 
    scarborough_data_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=IPFKGSTEYOILEGG2MJRFNKDX2YV5WEH44MR2KZDKZOWFBXWM&client_secret=21MX2NLSSB4PWVFR3Y04NYJJOQCNDEWTM0TMVVLVKXK2B3CD&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=700&limit=100'

In [140]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607f089fbf0a0a3f9a4fe9b4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.812986306300004,
    'lng': -79.18564005302258},
   'sw': {'lat': 43.80038629369999, 'lng': -79.20306674697744}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d669cba83865481c948fa53',
       'name': 'Images Salon & Spa',
       'location': {'address': '8130 Sheppard Ave E',
        'crossStreet': 'Morningside Ave',
        'lat': 43.80228301948931,
        'lng': -79.19856472801668,
        'labeledLatLngs':

### A6. Bring in 'get_category_type' function

In [256]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### A7. Review venue categories for each row in json file 

In [257]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-257-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Images Salon & Spa,Spa,43.802283,-79.198565
1,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,Tim Hortons,Coffee Shop,43.802000,-79.198169
4,Lee Valley,Hobby Shop,43.803161,-79.199681


In [258]:
print('{} venues in Scarborough were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues in Scarborough were returned by Foursquare.


### A8. Explore all neighborhoods in Toronto using the 'getNearbyVenues' function

In [259]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### A9. Call getNearbyVenues function to create a new dataframe toronto_venues

In [260]:
scarborough_venues = getNearbyVenues(names=scarborough_data['PostalCode'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )
print(scarborough_venues.shape)
scarborough_venues.head()

M1B
M1C
M1E
M1G
M1H
M1J
M1K
M1L
M1M
M1N
M1P
M1R
M1S
M1T
M1V
M1W
M1X
(95, 7)


,PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,M1E,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,M1E,43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


### A10. Explore venues in each Postal Code neighborhood

In [262]:
scarborough_venues.groupby('PostalCode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M1B,1,1,1,1,1,1
M1C,1,1,1,1,1,1
M1E,9,9,9,9,9,9
M1G,3,3,3,3,3,3
M1H,8,8,8,8,8,8
M1J,2,2,2,2,2,2
M1K,6,6,6,6,6,6
M1L,10,10,10,10,10,10
M1M,2,2,2,2,2,2


In [263]:
print('There are {} unique venue categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 57 unique venue categories.


## B. Analyze each neighborhood in Scarborough

### B1. Convert categorical data into integer using one hot encoding

In [264]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['PostalCode'] = scarborough_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,PostalCode,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Donut Shop,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,M1B,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M1C,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,M1E,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [265]:
scarborough_onehot.shape  ## we have number of venues and unique categories

(95, 58)

In [266]:
# We group the neighborhood by taking the mean of the frequency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('PostalCode').mean().reset_index()
scarborough_grouped

,PostalCode,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Donut Shop,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean BBQ Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant
0,M1B,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,M1C,0.0,0.000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,M1E,0.0,0.000,0.000000,0.000000,0.111111,0.0,0.111111,0.000000,0.0,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.111111,0.111111,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.111111,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.111111,0.0,0.111111,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.111111,0.111111,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,M1G,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.000000,0.666667,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.333333,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,M1H,0.0,0.125,0.000000,0.125000,0.125000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.125,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.125000,0.125000,0.0,0.125,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.125000,0.000000,0.000000,0.000000
5,M1J,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.5,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.50,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,M1K,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.166667,0.0,0.000,0.166667,0.166667,0.0,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.166667,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.000000
7,M1L,0.0,0.000,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.2,0.100000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0

In [267]:
scarborough_grouped.shape # we have 98 neighborhood after grouping

(16, 58)

### B2. We print each postal code's top 10 most common venues

In [268]:
num_top_venues = 10

for hood in scarborough_grouped['PostalCode']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['PostalCode'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                       venue  freq
0       Fast Food Restaurant   1.0
1        American Restaurant   0.0
2                  Pet Store   0.0
3  Latin American Restaurant   0.0
4         Light Rail Station   0.0
5                     Lounge   0.0
6             Medical Center   0.0
7              Metro Station   0.0
8         Mexican Restaurant   0.0
9  Middle Eastern Restaurant   0.0


----M1C----
                       venue  freq
0                        Bar   1.0
1        American Restaurant   0.0
2                   Pharmacy   0.0
3  Latin American Restaurant   0.0
4         Light Rail Station   0.0
5                     Lounge   0.0
6             Medical Center   0.0
7              Metro Station   0.0
8         Mexican Restaurant   0.0
9  Middle Eastern Restaurant   0.0


----M1E----
                 venue  freq
0    Electronics Store  0.11
1           Restaurant  0.11
2           Donut Shop  0.11
3       Medical Center  0.11
4                 Bank  0.11
5         Inters

### B3. Create a dataframe to store the top 10 most common venues in each postal code

In [271]:
# Function to sort the venues in decending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [272]:
# Create the dataframe

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = scarborough_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], 
                                                                          num_top_venues)

neighborhoods_venues_sorted.head()

IndexError: single positional indexer is out-of-bounds

In [273]:
neighborhoods_venues_sorted.shape

(16, 11)

### B4. Cluster postal code

In [275]:
#Run K-means to cluster the neighborhood into 5 clusters
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([2, 0, 1, 3, 1, 1, 1, 1, 4, 1])

In [276]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

scarborough_merged.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Intersection,Ice Cream Shop,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Farm
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Bar,Vietnamese Restaurant,College Stadium,Ice Cream Shop,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Donut Shop,Intersection,Mexican Restaurant,Electronics Store,Bank,Medical Center,Breakfast Spot,Restaurant,Rental Car Location,Fast Food Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3.0,Coffee Shop,Korean BBQ Restaurant,College Stadium,Ice Cream Shop,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Bakery,Bank,Caribbean Restaurant,Gas Station,Electronics Store,Farm


In [277]:
scarborough_merged.shape

(17, 16)

### B5. Visualize the cluster

In [285]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], 
                                  scarborough_merged['PostalCode'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

ValueError: cannot convert float NaN to integer

### B5a Cluster 1

In [279]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0.0,Bar,Vietnamese Restaurant,College Stadium,Ice Cream Shop,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


### B5b Cluster 2

In [280]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1.0,Donut Shop,Intersection,Mexican Restaurant,Electronics Store,Bank,Medical Center,Breakfast Spot,Restaurant,Rental Car Location,Fast Food Restaurant
4,Scarborough,1.0,Hakka Restaurant,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Bakery,Bank,Caribbean Restaurant,Gas Station,Electronics Store,Farm
5,Scarborough,1.0,Jewelry Store,Playground,Vietnamese Restaurant,Coffee Shop,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
6,Scarborough,1.0,Coffee Shop,Bus Station,Hobby Shop,Department Store,Chinese Restaurant,Train Station,Bar,Electronics Store,Indian Restaurant,Ice Cream Shop
7,Scarborough,1.0,Bus Line,Bakery,Park,Metro Station,Ice Cream Shop,Bus Station,Intersection,Soccer Field,Bar,Donut Shop
9,Scarborough,1.0,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Ice Cream Shop,Hobby Shop,Hakka Restaurant,Gas Station
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Brewery,Light Rail Station,Chinese Restaurant,Pet Store,Bar,Donut Shop,Hobby Shop,Hakka Restaurant
11,Scarborough,1.0,Middle Eastern Restaurant,Vietnamese Restaurant,Bakery,Smoke Shop,Sandwich Place,Auto Garage,Hakka Restaurant,General Entertainment,College Stadium,Gas Station
12,Scarborough,1.0,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,College Stadium,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station
13,Scarborough,1.0,Fast Food Restaurant,Pizza Place,Gas Station,Noodle House,Pharmacy,Fried Chicken Joint,Italian Restaurant,Bank,Thai Restaurant,Intersection


### B5c Cluster 3

In [281]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Vietnamese Restaurant,Intersection,Ice Cream Shop,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Farm


### B5d Cluster 4

In [282]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3.0,Coffee Shop,Korean BBQ Restaurant,College Stadium,Ice Cream Shop,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


### B5e Cluster 5

In [283]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,4.0,American Restaurant,Motel,College Stadium,Ice Cream Shop,Hobby Shop,Hakka Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
